In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install simplejson

     |████████████████████████████████| 130 kB 9.2 MB/s 


In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import xml.etree.ElementTree as ET
import json
import requests
import urllib
import urllib.request
import datetime
import folium
import os
import simplejson
import requests

# 주산지 선정 및 관측지점 mapping

## 2021년 1~8월 데이터를 기준으로 주산지 선정

In [4]:
!ls /content/drive/MyDrive/Capstone/private_data/AT_TSALET_ALL

AT_TSALET_ALL_202011.csv  AT_TSALET_ALL_202103.csv  AT_TSALET_ALL_202107.csv
AT_TSALET_ALL_202012.csv  AT_TSALET_ALL_202104.csv  AT_TSALET_ALL_202108.csv
AT_TSALET_ALL_202101.csv  AT_TSALET_ALL_202105.csv
AT_TSALET_ALL_202102.csv  AT_TSALET_ALL_202106.csv


In [9]:
filename_list = os.listdir('/content/drive/MyDrive/Capstone/private_data/AT_TSALET_ALL')

In [10]:
filename_list

['AT_TSALET_ALL_202011.csv',
 'AT_TSALET_ALL_202102.csv',
 'AT_TSALET_ALL_202108.csv',
 'AT_TSALET_ALL_202101.csv',
 'AT_TSALET_ALL_202012.csv',
 'AT_TSALET_ALL_202103.csv',
 'AT_TSALET_ALL_202104.csv',
 'AT_TSALET_ALL_202105.csv',
 'AT_TSALET_ALL_202106.csv',
 'AT_TSALET_ALL_202107.csv']

In [11]:
#filename_list = !ls /content/drive/MyDrive/Capstone/private_data/AT_TSALET_ALL ----------> private_data(train) 만드는데 사용한 데이터 AT_TSALET_ALL
filename_list_2019 = [] 
# filename_etc = []   

for filename in filename_list :
  if '2021' in filename :
    filename_list_2019.append(filename)

path = '/content/drive/MyDrive/Capstone/private_data/AT_TSALET_ALL/'
df_list = []
for filename in tqdm(filename_list_2019) :
    df = pd.read_csv(path + filename, engine='python')
    df_list.append(df)

data = pd.concat(df_list, sort=False).reset_index(drop=True)

100%|██████████| 8/8 [04:53<00:00, 36.64s/it]


## 품목 품종별 주산지 mapping

In [12]:
unique_pum = [
    '배추', '무', '양파', '건고추','마늘',
    '대파', '얼갈이배추', '양배추', '깻잎',
    '시금치', '미나리', '당근',
    '파프리카', '새송이', '팽이버섯', '토마토',
]

unique_kind = [
    '청상추', '백다다기', '애호박', '캠벨얼리', '샤인마스캇'
]

In [13]:
joosan_dict = dict()

# 품종별 주산지 mapping
for pum in tqdm(unique_pum) :
    pum_df = data[data['PUM_NM']==pum]
    joosan_list = pum_df.groupby(['SAN_NM'])['TOT_QTY'].sum().sort_values(ascending=False).index
    for i in range(10) :
        if joosan_list[i][-1] in ['군','구','도','시'] : # 국산만 골라내기
            joosan_dict[pum] = joosan_list[i]
            break
       
# 품종별 주산지 mapping
for pum in tqdm(unique_kind) :
    pum_df = data[data['KIND_NM']==pum]
    joosan_list = pum_df.groupby(['SAN_NM'])['TOT_QTY'].sum().sort_values(ascending=False).index
    for i in range(10) :
        if joosan_list[i][-1] in ['군','구','도','시'] : # 국산만 골라내기
            joosan_dict[pum] = joosan_list[i]
            break

100%|██████████| 5/5 [00:06<00:00,  1.37s/it]


In [ ]:
unique_pum = [ #----------------------------------------------------------------------> 10개 품목
    '배추', '무', '양파', '마늘', '대파', 
    '깻잎', '시금치', '미나리', '파프리카', '토마토',
]

In [ ]:
joosan_dict = dict() #----------------------------------------------------------------------> 10개 품목

# 품종별 주산지 mapping
for pum in tqdm(unique_pum) :
    pum_df = data[data['PUM_NM']==pum]
    joosan_list = pum_df.groupby(['SAN_NM'])['TOT_QTY'].sum().sort_values(ascending=False).index
    for i in range(10) :
        if joosan_list[i][-1] in ['군','구','도','시'] : # 국산만 골라내기
            joosan_dict[pum] = joosan_list[i]
            break

100%|██████████| 10/10 [00:15<00:00,  1.50s/it]


In [14]:
joosan_dict

{'건고추': '경상북도',
 '깻잎': '경상남도 밀양시',
 '당근': '제주도 제주시',
 '대파': '전라남도 신안군',
 '마늘': '경상남도 창녕군',
 '무': '제주도 제주시',
 '미나리': '경기도 시흥시',
 '배추': '전라남도 해남군',
 '백다다기': '충청남도 천안시',
 '새송이': '충청남도 천안시',
 '샤인마스캇': '경상북도 김천시',
 '시금치': '경기도 포천시',
 '애호박': '경상남도 진주시',
 '양배추': '제주도 제주시',
 '양파': '전라남도 무안군',
 '얼갈이배추': '경기도 포천시',
 '청상추': '전북 남원시',
 '캠벨얼리': '전북 남원시',
 '토마토': '부산 강서구',
 '파프리카': '강원도 철원군',
 '팽이버섯': '경상북도 청도군'}

## 주산지 - 관측지점 mapping

### 주산지의 위도 경도 구하기
카카오맵 API (Geocoding) - https://developers.kakao.com/product/map

kakao developers 회원가입 -> 내 애플리케이션 만들기 (임의대로) -> 플랫폼 -> WEB 등록 http://localhost:3000 -> Redirect URI 등록 	
https://colab.research.google.com/drive/1ZTgEBrEXmiFQ8eMnz8lZHFYS2zBmg4rs
-> 도구에서 REST API에서 토큰 발급 -> 내 애플리케이션에서 REST API 키 확인하기 => 순서 다를 수 있으나, 이 모든걸 완료 후에 키 사용 가능한듯

Kakao_ServiceKey.txt 메모장에 본인의 키를 작성하기
KakaoAK 인증키 이렇게 작성하여 사용할 것


In [ ]:
#주산지 정보들(코드 다시 돌리기 힘들때 사용하기 위해 미리 저장해둠)
joosan_dic = [{
              '건고추': '상북도', 
              '깻잎': '경상남도 밀양시',
              '당근': '제주도 제주시',
              '대파': '전라남도 신안군',
              '마늘': '경상남도 창녕군',
              '무': '제주도 제주시',
              '미나리': '경기도 시흥시',
              '배추': '전라남도 해남군',
              '백다다기': '충청남도 천안시',
              '새송이': '충청남도 천안시',
              '샤인마스캇': '경상북도 김천시',
              '시금치': '경기도 포천시',
              '애배추': '제주도 제주시',
              '양파': '전라남도 무안군',
              '얼갈이배호박': '경상남도 진주시',
              '양추': '경기도 포천시',
              '청상추': '전북 남원시',
              '캠벨얼리': '전북 남원시',
              '토마토': '부산 강서구',
              '파프리카': '강원도 철원군',
              '팽이버섯': '경상북도 청도군'
}]

In [ ]:
#주산지 정보들(코드 다시 돌리기 힘들때 사용하기 위해 미리 저장해둠) ---------------------> 이거 품목 축소한거 사용
joosan_dict = {
              '깻잎': '경상남도 밀양시',
              '대파': '전라남도 신안군',
              '마늘': '경상남도 창녕군',
              '무': '제주도 제주시',
              '미나리': '경기도 시흥시',
              '배추': '전라남도 해남군',
              '시금치': '경기도 포천시',
              '양파': '전라남도 무안군',
              '토마토': '부산 강서구',
              '파프리카': '강원도 철원군'         
}

In [15]:
y = [] #위도
x = [] #경도

Kakao_ServiceKey = open('/content/drive/MyDrive/Capstone/주산지, 날씨 코드/Kakao_ServiceKey.txt', 'r').read()
headers = {"Authorization": Kakao_ServiceKey}
for san in tqdm(joosan_dict.values()) :
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + san
    result = json.loads(str(requests.get(url, headers=headers).text))
    match_first = result['documents'][0]['address']
    y.append(float(match_first['y'])) #위도
    x.append(float(match_first['x'])) #경도
joosan_xy = pd.DataFrame({'SAN_NM' : joosan_dict.values(), 'y' : y, 'x' : x}).reset_index()

100%|██████████| 21/21 [00:12<00:00,  1.71it/s]


In [16]:
joosan_xy

,index,SAN_NM,y,x
0,0,전라남도 해남군,34.573517,126.599270
1,1,제주도 제주시,33.499534,126.531171
2,2,전라남도 무안군,34.990341,126.481705
3,3,경상북도,36.575996,128.505799
4,4,경상남도 창녕군,35.544615,128.492168
5,5,전라남도 신안군,34.833643,126.351408
6,6,경기도 포천시,37.894993,127.200333
7,7,제주도 제주시,33.499534,126.531171
8,8,경상남도 밀양시,35.503817,128.746608
9,9,경기도 포천시,37.894993,127.200333


### 농업기상관측지점정보
http://weather.rda.go.kr/weather/observationInfo.jsp

In [17]:
stn_info = pd.read_csv('/content/drive/MyDrive/Capstone/주산지, 날씨 코드/RDA_SPOT_INFO.csv')
stn_info['관측시작일'] = pd.to_datetime(stn_info['관측시작일'])
stn_info = stn_info[stn_info['관측시작일'].dt.year<=2015]

In [18]:
stn_info

,도명,지점명,지점코드,위도,경도,고도,설치주소,관측시작일
0,경기도,가평군 가평읍,477802A001,37.84621,127.50063,80.0,경기도 가평군 가평읍 아랫마장길 59,2011-05-22
1,경기도,고양시 구산동,411801A001,37.67453,126.70070,24.0,경기도 고양시 일산서구 구산동 1942,2010-12-06
2,경기도,고양시 덕양구,412040A002,37.64918,126.87036,39.0,경기도 고양시 덕양구 고양대로 1695 (원흥동),2013-08-24
3,경기도,광주시 목현동,464030A001,37.43231,127.23394,91.0,경기도 광주시 이배재로 209-5,2010-11-01
4,경기도,김포시 월곶면,415743A001,37.69489,126.55614,43.0,경기도 김포시 월곶면 오리정로 13,2010-12-14
...,...,...,...,...,...,...,...,...
205,부산,부산시 강서구,618803A001,35.20937,128.96128,3.0,경남 부산시 강서구 대저1동 2038,2010-12-20
206,인천,옹진군 백령면,409911A001,37.95742,124.71063,11.0,인천광역시 옹진군 백령면 진촌리 2359-4,2014-12-18
207,인천,옹진군 영흥면,409871A001,37.25288,126.46034,11.0,인천광역시 옹진군 영흥면 영흥남로435번길 29,2012-05-26
208,울산,울산시 청량면,689861A001,35.52411,129.26704,51.0,울산광역시 울주군 청량면 청량천변로 211,2009-03-02


### 주산지별로 농업기상관측지점과 거리 기준으로 mapping

In [19]:
y_san = joosan_xy['y']
x_san = joosan_xy['x']
y_stn = stn_info['위도']
x_stn = stn_info['경도']
code_dict = dict()

for y_san, x_san, san_name in zip(joosan_xy['y'], joosan_xy['x'], joosan_xy['SAN_NM']) :
    min_distance = 1000 #임의로 초기값 설정
    for y_stn, x_stn, stn_code in zip(stn_info['위도'], stn_info['경도'], stn_info['지점코드']) :
        distance = ((y_san-y_stn)**2 + (x_san-x_stn)**2)**0.5 #거리
        if distance < min_distance :
            min_distance = distance 
            stn_nearby = str(stn_code)
    code_dict[san_name] = stn_nearby

In [20]:
code_dict

{'강원도 철원군': '269811A001',
 '경기도 시흥시': '429843A001',
 '경기도 포천시': '487915A001',
 '경상남도 밀양시': '627911A001',
 '경상남도 진주시': '660985B001',
 '경상남도 창녕군': '635821A001',
 '경상북도': '757802A001',
 '경상북도 김천시': '037268B004',
 '경상북도 청도군': '714902A001',
 '부산 강서구': '618803A001',
 '전라남도 무안군': '5.35E+06',
 '전라남도 신안군': '535812A001',
 '전라남도 해남군': '536824B002',
 '전북 남원시': '590823A001',
 '제주도 제주시': '063057B009',
 '충청남도 천안시': '330846A001'}

### 농업기상관측지점 및 산지 분포 시각화

In [ ]:
stn_location = stn_info[['지점명','위도', '경도']]
san_location = joosan_xy

map = folium.Map(location = [36, 128], zoom_start =7)

# 농업기상관측지점 분포 (파랑)
for index in stn_location.index:
    stn_latitude = stn_location.loc[index,"위도"]
    stn_longtitude = stn_location.loc[index,"경도"]
    stn_tooltip = stn_location.loc[index,'지점명']
    folium.Marker([stn_latitude, stn_longtitude], popup = '('+str(stn_latitude)+', '+str(stn_longtitude)+')', tooltip = stn_tooltip).add_to(map) 

# 산지 분포 (빨강)    
for index in san_location.index:    
    san_latitude = san_location.loc[index,"y"]
    san_longtitude = san_location.loc[index,"x"]
    san_tooltip = san_location.loc[index,'SAN_NM']    
    folium.Marker([san_latitude, san_longtitude], popup = '('+str(san_latitude)+', '+str(san_longtitude)+')', tooltip = san_tooltip, icon = folium.Icon(color = 'red')).add_to(map)
    
map

# 농업기상데이터 API - 월별 일 기본 관측데이터 조회
관측년도, 관측월, 관측지점명, 관측지점코드 값으로 조회  
본 대회의 내용으로 운영계정 신청 가능 (신청 후 바로 승인되며, 일일 트래픽 10000 이용가능)  
https://www.data.go.kr/iim/api/selectAPIAcountView.do

위 링크로 접속하여 회원가입 후, 신청하면 인증 키 발급해주며, encoding 인증키를 CropWeather_ServiceKey.txt에 저장하여 사용


In [ ]:
CropWeather_ServiceKey = open('/content/drive/MyDrive/Capstone/주산지, 날씨 코드/CropWeather_ServiceKey.txt', 'r').read()
year_list = ['2015','2016', '2017', '2018', '2019', '2020']
month_list = ['01','02','03','04','05','06','07','08','09','10','11','12']
weather = pd.DataFrame()
first_run = 0
code_list = code_dict.values() # 주산지에 mapping 된 지점에 대해서만 조회
year_error, month_error, stn_code_error, url_error, f_obs_date_error = [],[],[],[],[]
colname_dict = dict()

for stn_code in tqdm(code_list) :
    for year in year_list :
        for month in month_list :
            url = 'http://apis.data.go.kr/1390802/AgriWeather/WeatherObsrInfo/GnrlWeather/getWeatherMonDayList?'
            params = {
                'serviceKey' : CropWeather_ServiceKey, #인증키
                'Page_No' : '1', # 페이지 번호
                'Page_Size' : '31', # 한 페이지 결과 수(1~100) (31일 이내 전체 표기)
                'search_Year' : year, # 관측년도
                'search_Month' : month, #관측월
                'obsr_Spot_Code' : stn_code # 관측지점코드
            }

            # url에 params 적용하기(붙이기)
            for key, value in zip(params.keys(), params.values()):
                if key == 'serviceKey' :
                    url = url + key +'=' + value
                else :
                    url = url + '&' + key + '=' + value
            
            try :
                response = urllib.request.urlopen(url).read()
                response_string = ET.fromstring(response)

                # response - header(0) / body(1) - ...items(3) - item(0) 
                items = response_string[1][3]

                if first_run == 0 :
                    for i in items[0] :
                        colname_dict[i.tag] = [] # {'no' : [], 'stn_Code' : [], ...} 
                        first_run += 1

                # 일자별로 반복 실행
                num_days = len(items) #28 or 30 or 31
                for index in range(num_days) : 
                    # 해당하는 리스트에 원소 넣기
                    for i in items[index] :
                        colname_dict[i.tag].append(i.text)
            except :
                year_error.append(year)
                month_error.append(month)
                stn_code_error.append(stn_code)
                url_error.append(url)
                f_obs_date_error.append(stn_info[stn_info['지점코드']==stn_code].reset_index()['관측시작일'][0])
               
            
# DataFrame에 값 채워넣기             
for col in colname_dict.keys() :
    weather[col] = colname_dict[col]

# 에러 발생한 요청 모음    
error = pd.DataFrame({'year': year_error, 
                      'month': month_error,
                      'stn_code': stn_code_error,
                      'url': url_error,  
                      '관측시작일': f_obs_date_error})

100%|██████████| 10/10 [01:51<00:00, 11.15s/it]


In [ ]:
CropWeather_ServiceKey = open('/content/drive/MyDrive/Capstone/주산지, 날씨 코드/CropWeather_ServiceKey.txt', 'r').read()
year_list = ['2020', '2021'] #---------------------------------------------------> 여기 건드림
month_list = ['01','02','03','04','05','06','07', '08','09','10','11','12']
weather2 = pd.DataFrame()
first_run = 0
code_list = code_dict.values() # 주산지에 mapping 된 지점에 대해서만 조회
year_error, month_error, stn_code_error, url_error, f_obs_date_error = [],[],[],[],[]
colname_dict = dict()

for stn_code in tqdm(code_list) :
    for year in year_list :
        for month in month_list :
            url = 'http://apis.data.go.kr/1390802/AgriWeather/WeatherObsrInfo/GnrlWeather/getWeatherMonDayList?'
            params = {
                'serviceKey' : CropWeather_ServiceKey, #인증키
                'Page_No' : '1', # 페이지 번호
                'Page_Size' : '31', # 한 페이지 결과 수(1~100) (31일 이내 전체 표기)
                'search_Year' : year, # 관측년도
                'search_Month' : month, #관측월
                'obsr_Spot_Code' : stn_code # 관측지점코드
            }

            # url에 params 적용하기(붙이기)
            for key, value in zip(params.keys(), params.values()):
                if key == 'serviceKey' :
                    url = url + key +'=' + value
                else :
                    url = url + '&' + key + '=' + value
            
            try :
                response = urllib.request.urlopen(url).read()
                response_string = ET.fromstring(response)

                # response - header(0) / body(1) - ...items(3) - item(0) 
                items = response_string[1][3]

                if first_run == 0 :
                    for i in items[0] :
                        colname_dict[i.tag] = [] # {'no' : [], 'stn_Code' : [], ...} 
                        first_run += 1

                # 일자별로 반복 실행
                num_days = len(items) #28 or 30 or 31
                for index in range(num_days) : 
                    # 해당하는 리스트에 원소 넣기
                    for i in items[index] :
                        colname_dict[i.tag].append(i.text)
            except :
                year_error.append(year)
                month_error.append(month)
                stn_code_error.append(stn_code)
                url_error.append(url)
                f_obs_date_error.append(stn_info[stn_info['지점코드']==stn_code].reset_index()['관측시작일'][0])
               
            
# DataFrame에 값 채워넣기             
for col in colname_dict.keys() :
    weather[col] = colname_dict[col]

# 에러 발생한 요청 모음    
error = pd.DataFrame({'year': year_error, 
                      'month': month_error,
                      'stn_code': stn_code_error,
                      'url': url_error,  
                      '관측시작일': f_obs_date_error})

100%|██████████| 10/10 [00:46<00:00,  4.65s/it]


In [ ]:
weather2

""


In [ ]:
error

,year,month,stn_code,url,관측시작일
0,2021,12,627911A001,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2010-09-06
1,2021,12,535812A001,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2011-09-30
2,2021,12,635821A001,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2014-06-10
3,2021,12,063057B009,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2014-05-01
4,2020,05,429843A001,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2010-11-05
...,...,...,...,...,...
139,2021,08,269811A001,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2010-12-16
140,2021,09,269811A001,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2010-12-16
141,2021,10,269811A001,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2010-12-16
142,2021,11,269811A001,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2010-12-16


In [ ]:
# 중복 제거
weather2 = weather2.drop_duplicates().reset_index(drop=True)

In [ ]:
# dtype변환(object --> float)
for col in weather2.columns[4:] :
    weather2[col] = weather2[col].astype(float)

In [ ]:
weather2

In [ ]:
weather.to_csv('/content/drive/MyDrive/Capstone/주산지, 날씨 코드/weather.csv', index=False)

# 전처리 (기상변수로 추가)
30일씩 12쿼터로 나누어서 평균 기온, 평균 습도, 누적 강수량, 이상 기후 누적 일수 등 추가  

In [ ]:
first_date = datetime.datetime.strptime('2021-07-31', '%Y-%m-%d') - datetime.timedelta(360) 
date_list = [] 
for delta in range(360) :
    date = first_date + datetime.timedelta(days = delta)
    date = datetime.datetime.strftime(date, '%Y-%m-%d')
    date_list.append(date)
date_df = pd.DataFrame({'date' : date_list})
train = pd.read_csv('/content/drive/MyDrive/Capstone/Data/ten_pum_train.csv')
train2 = pd.concat([date_df, train], sort = False).reset_index(drop=True) #2015~2020-09-28 -> 2020/11/6 - 2021/8/23

In [ ]:
first_date = datetime.datetime.strptime('2021-07-31', '%Y-%m-%d') - datetime.timedelta(2038)
date_list = [] 
for delta in range(2038) :
    date = first_date + datetime.timedelta(days = delta)
    date = datetime.datetime.strftime(date, '%Y-%m-%d')
    date_list.append(date)
date_df = pd.DataFrame({'date' : date_list})
train = pd.read_csv('/content/drive/MyDrive/Capstone/Data/ten_pum_train.csv')
train2 = pd.concat([date_df, train], sort = False).reset_index(drop=True) #2015~2020-09-28

In [ ]:
def weather_feature(temp_df, train2, date_df, allweather, pum, joosan_dict, code_dict, quater_days=30, num_quaters=12) : #290일로 바뀜
    # 2015년도 일자 추가
    temp_df = train2[['date',f'{pum}_거래량(kg)', f'{pum}_가격(원/kg)']]
    
    # 품종과 주산지 날씨 mapping
    joosanji = joosan_dict[pum]
    joosan_code = code_dict[joosanji]
    joosan_weather = allweather[allweather['stn_Code']==joosan_code].reset_index(drop=True)
    end_index = np.where(joosan_weather['date']=='2021-07-31')[0][0]
    joosan_weather = joosan_weather.iloc[:end_index+1] #2020-09-28 까지만 자르기
    temp_df = temp_df.merge(joosan_weather, on='date', how='left')

    # weather feature 추가
    col_list = temp_df.columns[6:]
    for num in range(1,num_quaters+1) :
        for index in range(360, len(temp_df)) :
            temp_quater_df = temp_df.iloc[index-quater_days*num : index-quater_days*(num-1)] #
            quater_temp = temp_quater_df['temp']
            temp_df.loc[index, f'rain_sum_{num}q'] = temp_quater_df['rain'].sum() # 누적 강수량
            temp_df.loc[index, f'heavy_rain_count_{num}q'] = np.where(temp_quater_df['rain']>90, 1, 0).sum() # 평균 강수량 90mm 이상 누적 일수
            temp_df.loc[index, f'low_temp_count_{num}q'] = np.where(quater_temp<5, 1, 0).sum() # 일평균 기온 5도 이하 누적 일수
            temp_df.loc[index, f'middle_temp_count_{num}q'] = np.where(((quater_temp>15)&(quater_temp<22)), 1, 0).sum() # 일평균 기온 15~22도 누적 일수
            temp_df.loc[index, f'high_temp_count_{num}q'] = np.where(quater_temp>32, 1, 0).sum() # 일평균 기온 32도 이상 누적 일수
            for col in col_list :
                temp_df.loc[index, f'avg_{col}_{num}q'] = temp_quater_df[col].mean() # 각 기상 요소의 평균값
    
    drop_col_list = temp_df.columns[3:19]
    temp_df = temp_df.drop(drop_col_list, 1).reset_index(drop=True)
    temp_df = temp_df.iloc[360:].reset_index(drop=True)
    
    return temp_df

In [ ]:
# weather_feature 함수 예시 *고쳐따*
pum = '배추'
temp_df = train[['date',f'{pum}_거래량(kg)', f'{pum}_가격(원/kg)']]
weather_feature(temp_df, train2, date_df, weather, pum, joosan_dict, code_dict, quater_days=30, num_quaters=12)

IndexError: ignored